In [1]:
import trimesh
import numpy as np
from irregular_object_packing.mesh.transform import scale_and_center_mesh, scale_to_volume
from irregular_object_packing.mesh.utils import print_mesh_info
from irregular_object_packing.packing import initialize, nlc_optimisation as nlc, chordal_axis_transform as cat
from irregular_object_packing.packing.growth_based_optimisation import SimSettings, Optimizer

mesh_volume = 0.1
container_volume = 10
coverage_rate = 0.3

DATA_FOLDER = "../data/mesh/"
loaded_mesh = trimesh.load_mesh(DATA_FOLDER + "RBC_normal.stl")
print_mesh_info(loaded_mesh, "loaded mesh")
# trimesh.Scene([loaded_mesh]).show()
# Scale the mesh to the desired volume
original_mesh = scale_and_center_mesh(loaded_mesh, mesh_volume)
print_mesh_info(original_mesh, "scaled mesh")

container = trimesh.primitives.Sphere(radius=1, center=[0, 0, 0])
# container = trimesh.primitives.Cylinder(radius=1, height=1)
print_mesh_info(container, "original container")

container = scale_to_volume(container, container_volume)
print_mesh_info(container, "scaled container")

Mesh info loaded mesh: <trimesh.Trimesh(vertices.shape=(642, 3), faces.shape=(1280, 3))>, 
volume: 71.04896973139347, 
bounding box: [[35.089 37.963 35.089]
 [42.911 40.037 42.911]] 
center of mass: [39. 39. 39.]

Mesh info scaled mesh: <trimesh.Trimesh(vertices.shape=(642, 3), faces.shape=(1280, 3))>, 
volume: 0.10000000000000006, 
bounding box: [[-0.4383 -0.1162 -0.4383]
 [ 0.4383  0.1162  0.4383]] 
center of mass: [-0. -0. -0.]

Mesh info original container: <trimesh.primitives.Sphere>, 
volume: 4.1887902047863905, 
bounding box: [[-1. -1. -1.]
 [ 1.  1.  1.]] 
center of mass: [-0.  0.  0.]

Mesh info scaled container: <trimesh.primitives.Sphere>, 
volume: 9.999999999999996, 
bounding box: [[-1.3365 -1.3365 -1.3365]
 [ 1.3365  1.3365  1.3365]] 
center of mass: [0. 0. 0.]



In [2]:
settings = SimSettings(sample_rate=50, max_angle=1 / 12 * np.pi, max_t=None, initial_scale=0.1, i_max=2)
Optimizer = Optimizer([original_mesh], container, settings)

Optimizer.run()


Iteration: 100%|██████████| 2/2 [01:35<00:00, 47.85s/it, obj_id=29]      


In [3]:
from irregular_object_packing.packing.plots import plot_state, plot_step_comparison

iteration = 1
test_k = 0
cat_cell_mesh_0 = cat.cat_mesh_from_data(Optimizer.cat_log[0], test_k)
cat_cell_mesh_1 = cat.cat_mesh_from_data(Optimizer.cat_log[iteration], test_k)

plot_step_comparison(
    original_mesh,
    [Optimizer.transform_log[0][test_k], Optimizer.transform_log[iteration][test_k]],
    cat_cell_mesh_0,
    cat_cell_mesh_1,
)

# %%
test_k = 0
print(Optimizer.transform_log[0][test_k])
print(Optimizer.transform_data[test_k])


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[ 0.1         1.99849897 -2.38022961 -3.05599584  1.07926535 -0.28181582
  0.25609046]
[ 0.14233444  2.01826334 -2.11140045 -3.02325159  1.07926535 -0.25208199
  0.28707255]


In [4]:

# Plot of the final packing
import pyvista as pv
from irregular_object_packing.packing.plots import create_plot, plot_state

plot_state(Optimizer)


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)